In [1]:
import os
import sys
sys.path.append('/workspace/GREAT_ASV_system/train/')

import numpy as np
import torch


# import vox_model_bank
import model_bank
from read_data import *
from my_dataloader import *
# from sklearn.metrics import roc_curve


In [3]:
model_path = '/workspace/LOGS_OUTPUT/std_server5/train/resnet34_SAP_TAUG_vox/ckpt/vox1test_metric_saver_MINC.model'
model_id = 'Resnet34_SAP'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [2]:
component_dir = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/backend_exp/res34sap'
PLDA_DIM = 370
PLDA_DATA_NAME = 'plda_data'
TEST_DATA_NAME = 'test_data_370'
PLDA_PARA_NAME = 'plda_para_370'
SCORING_PLDA_NAME = 'score_plda_370'
SCORING_COSINE_NAME = 'score_cosine_370'
AUX_DATA_NAME = 'aux_data_370'
SCORING_AUX_COSINE = 'score_aux_cosine_370'
SCORING_AUX_PLDA = 'score_aux_plda_370'

In [4]:
if not os.path.isdir(component_dir):
    os.makedirs(component_dir)

# GPU multiprocess

In [9]:
from multiprocessing import Process, Manager

In [10]:
num_p = 28

In [11]:
train_list = '/Lun0/zhiyong/dataset/plda_full_data.csv'
# train_list = '/Lun0/zhiyong/dataset/plda_full_data_noVAD.csv'
# train_data = CSVDataSet(train_list)
train_data = PickleDataSet(train_list)

In [12]:
train_data_len = len(train_data)
num_per_process = (1276888 // num_p) + 1

In [13]:
manager = Manager()
class_list_new_m = manager.list()

In [14]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45604
45580


In [22]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
    if i < 26:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = model_bank.get_model(model_id, model_metric, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [25]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Proc 26:1
Proc 24:1
Proc 27:1
Proc 25:1
Proc 22:1
Proc 16:1
Proc 19:1
Proc 20:1
Proc 8:1
Proc 10:1
Proc 14:1
Proc 23:1
Proc 21:1
Proc 18:1
Proc 6:1
Proc 12:1
Proc 13:1
Proc 15:1
Proc 4:1
Proc 2:1
Proc 17:1
Proc 5:1
Proc 7:1
Proc 11:1
Proc 9:1
Proc 3:1
Proc 0:1
Proc 1:1
Proc 26:2
Proc 24:2
Proc 25:2
Proc 27:2
Proc 16:2
Proc 22:2
Proc 20:2
Proc 8:2
Proc 21:2
Proc 14:2
Proc 10:2
Proc 18:2
Proc 12:2
Proc 23:2
Proc 19:2
Proc 15:2
Proc 4:2
Proc 6:2
Proc 13:2
Proc 9:2
Proc 17:2
Proc 0:2
Proc 2:2
Proc 11:2
Proc 7:2
Proc 5:2
Pr

In [26]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

1276888

In [27]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [28]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

1276888

In [29]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [30]:
count

1276888

In [31]:
len(class_list_new)

7323

In [32]:
import pickle
out = component_dir+'/'+PLDA_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

In [20]:
# class_list_new = {}
# for i in class_list:
#     class_list_new[i[:-1]] = class_list[i]

# PLDA_FIN

In [22]:
# import os
# import sys
# sys.path.append('./train')

# import numpy as np
# import torch


# # import vox_model_bank
# from train import train_model_new
# from train.read_data import *
# from train.my_dataloader import *
# # from sklearn.metrics import roc_curve

In [33]:
from kaldi_plda import *
from kaldi_lda import *
import numpy as np

In [34]:
import pickle
with open(component_dir+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [37]:
# Substract global mean
global_mean = np.zeros(512)
num_utt = 0
for count, i in enumerate(class_list_new):
    num_utt += class_list_new[i].shape[0]
    global_mean += class_list_new[i].shape[0] * np.mean(class_list_new[i], axis=0)
    
global_mean = (1.0 / num_utt) * global_mean
print('Norm of mean:', np.linalg.norm(global_mean))

Norm of mean: 1.9046652386925422


In [38]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i] - global_mean

In [39]:
lda = LDA(lda_dim=PLDA_DIM, ivector_dim=512)
for i in class_list_new:
    lda.AccStats(class_list_new[i])
print('lda norm of global mean:', lda.GetGlobalMean()[1])

lda norm of global mean: 1.0363030848906236e-07


In [40]:
transform = lda.ComputeLdaTransform()

the input data has norm of mean 1.0363030848906236e-07
[7.05680318e+00 6.83084110e+00 6.49589130e+00 6.25103781e+00
 6.02699328e+00 5.93644421e+00 5.74553620e+00 5.71113328e+00
 5.52863863e+00 5.48530371e+00 5.33798738e+00 5.29171180e+00
 5.19417991e+00 5.13465376e+00 5.07992271e+00 5.03656835e+00
 4.94590764e+00 4.87222457e+00 4.86027973e+00 4.83695617e+00
 4.67446231e+00 4.62048504e+00 4.56140204e+00 4.53129694e+00
 4.49590328e+00 4.46416332e+00 4.37204092e+00 4.31959085e+00
 4.31300552e+00 4.24568159e+00 4.22761656e+00 4.13339006e+00
 4.09174010e+00 4.06669643e+00 4.00950966e+00 3.98957290e+00
 3.92033707e+00 3.90857520e+00 3.83256738e+00 3.74048687e+00
 3.67952474e+00 3.63867904e+00 3.60639986e+00 3.55344168e+00
 3.53629863e+00 3.48462131e+00 3.43491949e+00 3.41205987e+00
 3.38153426e+00 3.36127220e+00 3.34227493e+00 3.30985634e+00
 3.22345557e+00 3.20170408e+00 3.17643705e+00 3.10434679e+00
 3.08763803e+00 3.05436367e+00 3.01382095e+00 2.98352146e+00
 2.93198022e+00 2.90411634e+00

In [41]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i].dot(transform.T)

In [42]:
# normlize to sqrt(dim)
for i in class_list_new:
    scale = np.sqrt(PLDA_DIM) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
    class_list_new[i] = scale * class_list_new[i]

In [43]:
plda_stats = PldaStats(PLDA_DIM)
for i in class_list_new:
    plda_stats.add_samples(class_list_new[i])

In [44]:
plda_stats.sort()
plda_stats.is_sorted()

True

In [45]:
# test_fin_prior
plda_estimator = PldaEstimation(plda_stats)
plda_paras = plda_estimator.estimate(iteration=5)

1 5
nllr_x: 410.5065199581329
nllr_y: -371.40425287913286
normalized_nllr: 39.10226707900005
nllr_m: 411.2795475687941
nllr_m_2: 411.27954756879467
part1_residual -456.1316055350453
part2_mean -411.2795475687941
normlized_obj -455.8743773204432
2 5
nllr_x: 237.23228226515957
nllr_y: -457.65293252226996
normalized_nllr: -220.42065025711042
nllr_m: 243.78477531194216
nllr_m_2: 243.78477531194218
part1_residual -438.72547415264864
part2_mean -243.78477531194218
normlized_obj -437.60748202051917
3 5
nllr_x: 234.43450561162115
nllr_y: -457.5477137050012
normalized_nllr: -223.11320809338005
nllr_m: 242.64087607144768
nllr_m_2: 242.64087607144745
part1_residual -438.72783256888783
part2_mean -242.64087607144765
normlized_obj -437.60326660661804
4 5
nllr_x: 234.15237832022436
nllr_y: -457.4238974777612
normalized_nllr: -223.27151915753686
nllr_m: 242.6170850683921
nllr_m_2: 242.61708506839204
part1_residual -438.72782922989177
part2_mean -242.61708506839224
normlized_obj -437.603126844487
5 5


In [46]:
import pickle
out = component_dir+'/'+ PLDA_PARA_NAME
with open(out, 'wb') as handle:
    pickle.dump(plda_paras, handle)

In [47]:
import pickle
with open(component_dir+'/'+ PLDA_PARA_NAME, 'rb') as handle:
    plda_paras = pickle.load(handle)

In [48]:
plda = PLDA(plda_paras[0], plda_paras[1], plda_paras[2])

# Evaluation

In [6]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = '/workspace/DATASET/server1/dataset/trial_data_csv.csv'
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 256, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, None)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500


In [7]:
count+1

4874

In [8]:
len(test_list)

4874

In [9]:
import pickle
out = component_dir+'/'+TEST_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# aux trial

In [4]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = '/Lun0/zhiyong/dataset/trial_aux_data.csv'
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = train_model_new.get_model(model_id, model_metric, _, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using new training model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000


In [5]:
count+1

36237

In [6]:
len(test_list)

36237

In [9]:
import pickle
out = component_dir+'/'+AUX_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# sdsvc dev trial & cos scoring

In [54]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = '/Lun0/zhiyong/sdsvc_t2_small_sparse/trial_data_small_csv.csv'
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = train_model_new.get_model(model_id, model_metric, _, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using new training model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000


In [55]:
count+1

14490

In [56]:
len(test_list)

14490

In [57]:
import pickle
out = component_dir+'/'+'sdscv_trial_data'
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

In [58]:
trail_path = '/Lun0/zhiyong/sdsvc_t2_small_sparse/small_tiral_list'
score_out_path = component_dir+'/'+'sdsvc_cos'

In [59]:
import pickle
with open(component_dir+'/'+'sdscv_trial_data', 'rb') as handle:
    test_list = pickle.load(handle)

In [60]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [61]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [71]:
from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [63]:
score_file = component_dir+'/'+'sdsvc_cos'
key_file = '/Lun0/zhiyong/sdsvc_t2_small_sparse/small_tiral_list'

_ = scoring(score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	04.68	0.389	1.000


# Scoring

In [53]:
trail_path = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
score_out_path = component_dir+'/'+SCORING_PLDA_NAME

In [54]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [55]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda.transform_ivector(test_list[i], 1)

In [56]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [47]:
# Cosine Scoring

In [18]:
trail_path = '/workspace/DATASET/server1/vox1_o_trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [19]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [20]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [21]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [22]:
print(count+1)

37611


In [14]:
trail_path = '/workspace/DATASET/server1/veri_test2.txt'
new_trial_path = '/workspace/DATASET/server1/vox1_o_trials'

with open(trail_path, 'r') as f:
    lines = f.readlines()

In [17]:
with open(new_trial_path, 'w') as f:
    for line in lines:
        cmp = line[:-1].split(' ')
        if cmp[0] == '0':
            cmp_a = cmp[1].split('/')
            out_1 = cmp_a[0]+'-'+cmp_a[1]+'-'+cmp_a[2]
            cmp_b = cmp[2].split('/')
            out_2 = cmp_b[0]+'-'+cmp_b[1]+'-'+cmp_b[2]
            out_3 = 'nontarget'
            out = out_1+' '+out_2+' '+out_3+'\n'
        else:
            cmp_a = cmp[1].split('/')
            out_1 = cmp_a[0]+'-'+cmp_a[1]+'-'+cmp_a[2]
            cmp_b = cmp[2].split('/')
            out_2 = cmp_b[0]+'-'+cmp_b[1]+'-'+cmp_b[2]
            out_3 = 'target'
            out = out_1+' '+out_2+' '+out_3+'\n'
        f.write(out)

In [ ]:
# Aux scoring PLDA

In [37]:
trail_path = '/Lun0/zhiyong/dataset/aux_tiral_list'
score_out_path = component_dir+'/'+SCORING_AUX_PLDA

In [38]:
import pickle
with open(component_dir+'/'+AUX_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [39]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = (np.sqrt(512) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
    test_list[i] = plda.transform_ivector(test_list[i], 1)

In [40]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = plda.log_likelihood_ratio(enroll_emb, 1, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id00562-X7FJ3M3bz3c-00127.wav id00562-9tqfxnIUShw-00071.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000


In [ ]:
# Aux scoring cosine

In [12]:
trail_path = '/Lun0/zhiyong/dataset/aux_tiral_list'
score_out_path = component_dir+'/'+SCORING_AUX_COSINE

In [13]:
import pickle
with open(component_dir+'/'+AUX_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [14]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [15]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id00562-X7FJ3M3bz3c-00127.wav id00562-9tqfxnIUShw-00071.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000


# scoring aux

In [17]:
from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [41]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_score_file = component_dir+'/'+ SCORING_AUX_PLDA
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'

_ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [aux_score_file])
applying(linear_model_pth, [score_file], calib_score_file)
_ = scoring(calib_score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	01.98	0.216	0.303
Starting point for CLLR is 0.703163
Converged linear model with loss 0.12720251288238166

Set	EER[%]	min_C	act_C
OUT	01.98	0.216	0.328


In [50]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_score_file = component_dir+'/'+ SCORING_AUX_COSINE
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'

_ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [aux_score_file])
applying(linear_model_pth, [score_file], calib_score_file)
_ = scoring(calib_score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	02.18	0.231	1.000
Starting point for CLLR is 0.844346
Converged linear model with loss 0.13304337922710377

Set	EER[%]	min_C	act_C
OUT	02.18	0.231	0.263


# En aux

In [42]:
score_file_1_aux = component_dir+'/'+SCORING_AUX_PLDA
score_file_2_aux = component_dir+'/'+SCORING_AUX_COSINE
score_file_1 = component_dir+'/'+SCORING_PLDA_NAME
score_file_2 = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = component_dir+'/'+'fuse_aux'
linear_model_pth = component_dir+'/'+'calib.pth'

# _ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [score_file_1_aux, score_file_2_aux])
applying(linear_model_pth, [score_file_1, score_file_2], calib_score_file)
_ = scoring(calib_score_file, key_file)

Starting point for CLLR is 0.773755
Converged linear model with loss 0.1185242436564276

Set	EER[%]	min_C	act_C
OUT	01.75	0.205	0.264


In [51]:
score_file_1_aux = component_dir+'/'+SCORING_AUX_PLDA+'_norm'
score_file_2_aux = component_dir+'/'+SCORING_AUX_COSINE+'_norm'
score_file_1 = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
score_file_2 = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = component_dir+'/'+'fuse_aux_norm'
linear_model_pth = component_dir+'/'+'calib.pth'

# _ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [score_file_1_aux, score_file_2_aux])
applying(linear_model_pth, [score_file_1, score_file_2], calib_score_file)
_ = scoring(calib_score_file, key_file)

Starting point for CLLR is 0.704718
Converged linear model with loss 0.23705743394532688

Set	EER[%]	min_C	act_C
OUT	01.66	0.285	0.454


In [52]:
score_file_1_aux = component_dir+'/'+SCORING_AUX_PLDA
score_file_2_aux = component_dir+'/'+SCORING_AUX_COSINE
score_file_3_aux = component_dir+'/'+SCORING_AUX_PLDA+'_norm'
score_file_4_aux = component_dir+'/'+SCORING_AUX_COSINE+'_norm'
score_file_1 = component_dir+'/'+SCORING_PLDA_NAME
score_file_2 = component_dir+'/'+SCORING_COSINE_NAME
score_file_3 = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
score_file_4 = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = component_dir+'/'+'fuse_aux_4'
linear_model_pth = component_dir+'/'+'calib.pth'

# _ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [score_file_1_aux, score_file_2_aux, score_file_3_aux, score_file_4_aux])
applying(linear_model_pth, [score_file_1, score_file_2, score_file_3, score_file_4], calib_score_file)
_ = scoring(calib_score_file, key_file)

Starting point for CLLR is 0.739236
Converged linear model with loss 0.09995077823341468

Set	EER[%]	min_C	act_C
OUT	01.61	0.185	0.195


# Score norm aux

In [44]:
from score_norm_1 import score_norm

In [59]:
ori_score = component_dir+'/'+SCORING_PLDA_NAME
norm_score = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
score_norm(ori_score, norm_score)

/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [45]:
ori_score = component_dir+'/'+SCORING_AUX_PLDA
norm_score = component_dir+'/'+SCORING_AUX_PLDA+'_norm'
score_norm(ori_score, norm_score)

/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [47]:
score_file = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_score_file = component_dir+'/'+ SCORING_AUX_PLDA+'_norm'
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'

_ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [aux_score_file])
applying(linear_model_pth, [score_file], calib_score_file)
_ = scoring(calib_score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	01.88	0.312	1.000
Starting point for CLLR is 0.698849
Converged linear model with loss 0.24405807569514407

Set	EER[%]	min_C	act_C
OUT	01.88	0.312	0.524


In [61]:
ori_score = component_dir+'/'+SCORING_COSINE_NAME
norm_score = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
score_norm(ori_score, norm_score)

/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [48]:
ori_score = component_dir+'/'+SCORING_AUX_COSINE
norm_score = component_dir+'/'+SCORING_AUX_COSINE+'_norm'
score_norm(ori_score, norm_score)

/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [49]:
score_file = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
aux_score_file = component_dir+'/'+ SCORING_AUX_COSINE+'_norm'
aux_key_file = '/Lun0/zhiyong/dataset/aux_tiral_list'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'

_ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, aux_key_file, [aux_score_file])
applying(linear_model_pth, [score_file], calib_score_file)
_ = scoring(calib_score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	01.81	0.311	1.000
Starting point for CLLR is 0.710586
Converged linear model with loss 0.26639713490619543

Set	EER[%]	min_C	act_C
OUT	01.81	0.311	0.635


# scoring

In [23]:
sys.path.append('/workspace/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [24]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/workspace/DATASET/server1/vox1_o_trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.018242740134028296, 0.19993050806978083, 1.0)
Starting point for CLLR is 0.833777
Converged linear model with loss 0.07151467749816884
(0.018242740134028296, 0.19993050806978083, 1.0)


In [62]:
sys.path.append('/Lun2/rzz/kaldi-master/egs/zhiyong/GREAT_ASV_system/score/')

from my_scorer import scoring
from calibrate_scores import calibrating
from apply_calibration import applying

In [65]:
score_file = component_dir+'/'+SCORING_PLDA_NAME
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(calib_score_file, key_file, config))

(0.02104984093319194, 0.26163838812301365, 0.4125662778366914)
Starting point for CLLR is 0.326006
Converged linear model with loss 0.0862339321029274
(0.02104984093319194, 0.26163838812301365, 0.2688229056203606)


In [66]:
score_file = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.02348886532343586, 0.24504241781548108, 1.0)
Starting point for CLLR is 0.842628
Converged linear model with loss 0.09200133308491727
(0.02348886532343586, 0.24504241781548108, 1.0)


# Score norm

In [58]:
from score_norm_1 import score_norm

In [59]:
ori_score = component_dir+'/'+SCORING_PLDA_NAME
norm_score = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
score_norm(ori_score, norm_score)

/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [60]:
score_file = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'

_ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
_ = scoring(calib_score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	01.88	0.312	1.000
Starting point for CLLR is 0.538934
Converged linear model with loss 0.07954009941343457

Set	EER[%]	min_C	act_C
OUT	01.88	0.312	0.334


In [61]:
ori_score = component_dir+'/'+SCORING_COSINE_NAME
norm_score = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
score_norm(ori_score, norm_score)

/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/dell/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [62]:
score_file = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'

_ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
_ = scoring(calib_score_file, key_file)


Set	EER[%]	min_C	act_C
OUT	01.81	0.311	1.000
Starting point for CLLR is 0.536303
Converged linear model with loss 0.07717334626909308

Set	EER[%]	min_C	act_C
OUT	01.81	0.311	0.331


# En

In [63]:
score_file_1 = component_dir+'/'+SCORING_PLDA_NAME
score_file_2 = component_dir+'/'+SCORING_COSINE_NAME
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = component_dir+'/'+'fuse'
linear_model_pth = component_dir+'/'+'calib.pth'

# _ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, key_file, [score_file_1, score_file_2])
applying(linear_model_pth, [score_file_1, score_file_2], calib_score_file)
_ = scoring(calib_score_file, key_file)

Starting point for CLLR is 0.581688
Converged linear model with loss 0.074283602014203

Set	EER[%]	min_C	act_C
OUT	01.74	0.204	0.221


In [65]:
score_file_1 = component_dir+'/'+SCORING_PLDA_NAME+'_norm'
score_file_2 = component_dir+'/'+SCORING_COSINE_NAME+'_norm'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = component_dir+'/'+'fuse_norm'
linear_model_pth = component_dir+'/'+'calib.pth'

# _ = scoring(score_file, key_file)
calibrating(linear_model_pth, 50, key_file, [score_file_1, score_file_2])
applying(linear_model_pth, [score_file_1, score_file_2], calib_score_file)
_ = scoring(calib_score_file, key_file)

Starting point for CLLR is 0.537618
Converged linear model with loss 0.0694378887070766

Set	EER[%]	min_C	act_C
OUT	01.61	0.266	0.291


# JB

In [68]:
from JB import *
import numpy as np

/home/dell/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [69]:
import pickle
with open(component_dir+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [70]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])

In [71]:
train_list = np.zeros((1276888, 512))
label = []
last_pos = 0
for count, i in enumerate(class_list_new):
    this_len = class_list_new[i].shape[0]
    train_list[last_pos:last_pos+this_len] = class_list_new[i]
    last_pos = last_pos+this_len
    label = label + this_len * [i]
    if (count+1) % 1000 == 0:
        print(count+1)

1000
2000
3000
4000
5000
6000
7000


In [75]:
train_list.shape

(1276888, 512)

In [76]:
len(label)

1276888

In [77]:
global_mean = np.mean(train_list, axis=0)

In [81]:
np.linalg.norm(global_mean)

1.9046652415934415

In [82]:
train_list_centered = train_list - global_mean

In [83]:
A, G = JointBayesian_Train(train_list_centered, label, fold = component_dir, complete_stats=True)

(1276888, 512)
prepare done, maxNumberInOneClass= 1002
Iterations-0: 0.0010711645996589437
Iterations-1: 1.3523100215732059e-05
Iterations-2: 2.4056012775033254e-06
Iterations-3: 7.342351828003286e-07
Iterations-4: 2.8068888850328114e-07
Iterations-5: 1.184689619041678e-07
Iterations-6: 5.267479095722711e-08
Iterations-7: 2.4178416589029948e-08
Iterations-8: 1.1338391238278522e-08
Iterations-9: 5.399002417915827e-09
Iterations-10: 2.6014852098979146e-09
Iterations-11: 1.2648995479417483e-09
Iterations-12: 6.188518191301809e-10
Iterations-13: 3.0677699544240897e-10
Iterations-14: 1.5350548541958987e-10
Iterations-15: 8.009531443608684e-11
Iterations-16: 4.7031017931828334e-11
Iterations-17: 3.413629428325939e-11
Iterations-18: 3.0268280274376596e-11
Iterations-19: 2.927871093020977e-11


In [84]:
trail_path = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
score_out_path = component_dir+'/'+'JB_score'

In [85]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [86]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
#     test_list[i] = transform.dot(test_list[i])
#     test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]
#     test_list[i] = plda.transform_ivector(test_list[i], 1)

In [88]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = Verify(A, G, enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [89]:
score_file = component_dir+'/'+'JB_score'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.028791092258748652, 0.2808589607635198, 0.592099681866384)
Starting point for CLLR is 10.192940
(0.028791092258748652, 0.2808589607635198, 0.592099681866384)


# LDA-JB

In [115]:
from JB_tmp import *
import numpy as np

In [116]:
import pickle
with open(component_dir+'/'+PLDA_DATA_NAME, 'rb') as handle:
    class_list_new = pickle.load(handle)

In [117]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])

In [118]:
# Substract global mean
global_mean = np.zeros(512)
num_utt = 0
for count, i in enumerate(class_list_new):
    num_utt += class_list_new[i].shape[0]
    global_mean += class_list_new[i].shape[0] * np.mean(class_list_new[i], axis=0)
    
global_mean = (1.0 / num_utt) * global_mean
print('Norm of mean:', np.linalg.norm(global_mean))

Norm of mean: 1.9046652386925422


In [119]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i] - global_mean

In [120]:
PLDA_DIM = 512

In [121]:
lda = LDA(lda_dim=PLDA_DIM, ivector_dim=512)
for i in class_list_new:
    lda.AccStats(class_list_new[i])
print('lda norm of global mean:', lda.GetGlobalMean()[1])

lda norm of global mean: 1.0363030848906236e-07


In [122]:
transform = lda.ComputeLdaTransform()

the input data has norm of mean 1.0363030848906236e-07
[7.05680318e+00 6.83084110e+00 6.49589130e+00 6.25103781e+00
 6.02699328e+00 5.93644421e+00 5.74553620e+00 5.71113328e+00
 5.52863863e+00 5.48530371e+00 5.33798738e+00 5.29171180e+00
 5.19417991e+00 5.13465376e+00 5.07992271e+00 5.03656835e+00
 4.94590764e+00 4.87222457e+00 4.86027973e+00 4.83695617e+00
 4.67446231e+00 4.62048504e+00 4.56140204e+00 4.53129694e+00
 4.49590328e+00 4.46416332e+00 4.37204092e+00 4.31959085e+00
 4.31300552e+00 4.24568159e+00 4.22761656e+00 4.13339006e+00
 4.09174010e+00 4.06669643e+00 4.00950966e+00 3.98957290e+00
 3.92033707e+00 3.90857520e+00 3.83256738e+00 3.74048687e+00
 3.67952474e+00 3.63867904e+00 3.60639986e+00 3.55344168e+00
 3.53629863e+00 3.48462131e+00 3.43491949e+00 3.41205987e+00
 3.38153426e+00 3.36127220e+00 3.34227493e+00 3.30985634e+00
 3.22345557e+00 3.20170408e+00 3.17643705e+00 3.10434679e+00
 3.08763803e+00 3.05436367e+00 3.01382095e+00 2.98352146e+00
 2.93198022e+00 2.90411634e+00

In [123]:
for i in class_list_new:
    class_list_new[i] = class_list_new[i].dot(transform.T)

In [124]:
# normlize to sqrt(dim)
for i in class_list_new:
    scale = np.sqrt(PLDA_DIM) / np.linalg.norm(class_list_new[i], axis=1, keepdims=True)
    class_list_new[i] = scale * class_list_new[i]

In [125]:
train_list = np.zeros((1276888, PLDA_DIM))
label = []
last_pos = 0
for count, i in enumerate(class_list_new):
    this_len = class_list_new[i].shape[0]
    train_list[last_pos:last_pos+this_len] = class_list_new[i]
    last_pos = last_pos+this_len
    label = label + this_len * [i]
    if (count+1) % 1000 == 0:
        print(count+1)

1000
2000
3000
4000
5000
6000
7000


In [126]:
train_list.shape

(1276888, 512)

In [127]:
len(label)

1276888

In [128]:
A, G = JointBayesian_Train(train_list, label, fold = component_dir, complete_stats=True)

(1276888, 512)
prepare done, maxNumberInOneClass= 1002
Iterations-0: 0.0013375771781835597
Iterations-1: 0.00020715537950710702
Iterations-2: 8.002533716710512e-05
Convergence:  2 8.002533716710512e-05


In [129]:
trail_path = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
score_out_path = component_dir+'/'+'JB_lda_512_score'

In [130]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [131]:
for i in test_list:
    test_list[i] = test_list[i].squeeze()
    test_list[i] = test_list[i] - global_mean
    test_list[i] = transform.dot(test_list[i])
    test_list[i] = (np.sqrt(PLDA_DIM) / np.linalg.norm(test_list[i])) * test_list[i]

In [132]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()

            cosine = Verify(A, G, enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000


In [110]:
score_file = component_dir+'/'+'JB_lda_score'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.020837751855779407, 0.26558854718982505, 0.5770943796394485)
Starting point for CLLR is 10.179847
(0.020837751855779407, 0.26558854718982505, 0.5770943796394485)


In [133]:
score_file = component_dir+'/'+'JB_lda_512_score'
key_file = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
calib_score_file = score_file+'_calib'
linear_model_pth = component_dir+'/'+'calib.pth'
config = {'p_target': [0.01, 0.005], 'c_miss': 1, 'c_fa': 1}

print(scoring(score_file, key_file, config))
calibrating(linear_model_pth, 50, key_file, [score_file])
applying(linear_model_pth, [score_file], calib_score_file)
print(scoring(score_file, key_file, config))

(0.01993637327677622, 0.26134676564156306, 0.5799575821845175)
Starting point for CLLR is 10.246580
(0.01993637327677622, 0.26134676564156306, 0.5799575821845175)
